<a href="https://colab.research.google.com/github/sungjin-kim-data/Hierarhical-Clustering/blob/master/Hierarhical_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# improt
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram

In [ ]:
import numpy as np
import pandas as pd
import math
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import StandardScaler

plt.style.use("default")

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
# 데이터 불러오기
df = pd.read_csv('/content/dataset.csv')
df.head()

In [ ]:
# 데이터 정제
#diagnosis columns M은 1, B는 0으로 바꾼 뒤, label이라는 변수에 저장
df = df.replace({'diagnosis','M'}, 1)
df = df.replace({'diagnosis','B'}, 0)

label = df['diagnosis']
label[:5]

In [ ]:
df = df.drop(['id', 'diagnosis', 'Unnamed: 32'], axis = 1)

In [ ]:
# AgglomerativeClustering(n_clusters=2, *, affinity='euclidean', memory=None, connectivity=None, compute_full_tree='auto', linkage='ward', distance_threshold=None, compute_distances=False)
ag = AgglomerativeClustering(n_clusters=2, compute_distances=True)
ag.fit(df)

In [ ]:
# zeros
counts = np.zeros(ag.children_.shape[0])
n_samples = len(ag.labels_)

In [ ]:
# 각 샘플 생성
for i, merge in enumerate(ag.children_):
  current_count = 0
  for child_idx in merge:
      if child_idx < n_samples:
          current_count += 1  # leaf node
      else:
          current_count += counts[child_idx - n_samples]
  counts[i] = current_count

In [ ]:
linkage_matrix = np.column_stack([ag.children_, ag.distances_, counts]).astype(float)

In [ ]:
dendrogram(linkage_matrix)
plt.title('Hierarhical Clustering')
plt.show()

### **다른 예시**

In [ ]:
# MNIST digit 이미지 중 20개의 이미지를 무작위로 골라 계층적 군집화를 적용

from sklearn.datasets import load_digits

digits = load_digits()
n_image = 20
np.random.seed(0)
idx = np.random.choice(range(len(digits.images)), n_image)
X = digits.data[idx]
images = digits.images[idx]

plt.figure(figsize=(12, 1))
for i in range(n_image):
    plt.subplot(1, n_image, i + 1)
    plt.imshow(images[i], cmap=plt.cm.bone)
    plt.grid(False)
    plt.xticks(())
    plt.yticks(())
    plt.title(i)

In [ ]:
from scipy.cluster.hierarchy import linkage, dendrogram

Z = linkage(X, 'ward')
Z

In [ ]:
# AgglomerativeClustering 클래스

from matplotlib.offsetbox import OffsetImage, AnnotationBbox

plt.figure(figsize=(10, 4))
ax = plt.subplot()

ddata = dendrogram(Z)

dcoord = np.array(ddata["dcoord"])
icoord = np.array(ddata["icoord"])
leaves = np.array(ddata["leaves"])
idx = np.argsort(dcoord[:, 2])
dcoord = dcoord[idx, :]
icoord = icoord[idx, :]
idx = np.argsort(Z[:, :2].ravel())
label_pos = icoord[:, 1:3].ravel()[idx][:20]

for i in range(20):
    imagebox = OffsetImage(images[i], cmap=plt.cm.bone_r, interpolation="bilinear", zoom=3)
    ab = AnnotationBbox(imagebox, (label_pos[i], 0),  box_alignment=(0.5, -0.1), 
                        bboxprops={"edgecolor" : "none"})
    ax.add_artist(ab)

plt.show()